In [69]:
import numpy as np
import plotly.graph_objects as go

class OptionLeg:
    def __init__(self, strike, price, quantity, option_type):
        self.strike = strike
        self.price = price
        self.quantity = quantity
        self.option_type = option_type.lower()

    def payoff(self, spot_prices):
        if self.option_type == 'call':
            intrinsic = np.maximum(spot_prices - self.strike, 0)
        elif self.option_type == 'put':
            intrinsic = np.maximum(self.strike - spot_prices, 0)
        else:
            raise ValueError("option_type must be 'call' or 'put'")
        return self.quantity * intrinsic

    def pnl(self, spot_prices):
        return self.payoff(spot_prices) - self.total_cost()

    def total_cost(self):
        return self.price * self.quantity

    def description(self):
        direction = "Long" if self.quantity > 0 else "Short"
        return f"{direction} {self.option_type.upper()} @ {self.strike}"


class StockLeg:
    def __init__(self, price, quantity):
        self.price = price
        self.quantity = quantity

    def payoff(self, spot_prices):
        return self.quantity * spot_prices

    def pnl(self, spot_prices):
        return self.payoff(spot_prices) - self.total_cost()

    def total_cost(self):
        return self.price * self.quantity

    def description(self):
        direction = "Long" if self.quantity > 0 else "Short"
        return f"{direction} Stock @ {self.price:.2f}"


class Portfolio:
    def __init__(self, exchange_rate = 1.0):
        self.legs = []
        self.exchange_rate = exchange_rate
        
    def add_leg(self, leg):
        self.legs.append(leg)

    def total_payoff(self, spot_prices):
        return sum(leg.payoff(spot_prices) for leg in self.legs)

    def total_pnl(self, spot_prices):
        return sum(leg.pnl(spot_prices) for leg in self.legs)

    def plot_pnl(self, spot_prices):
        fig = go.Figure()
        for leg in self.legs:
            fig.add_trace(go.Scatter(
                x=spot_prices,
                y=leg.pnl(spot_prices) * self.exchange_rate,
                mode='lines',
                name=leg.description()
            ))

        total_pnl = self.total_pnl(spot_prices) * self.exchange_rate
        
        fig.add_trace(go.Scatter(
            x=spot_prices,
            y=total_pnl,
            mode='lines',
            name='Total PnL',
            line=dict(width=4, dash='dash')
        ))

        max_loss = np.min(total_pnl)
        max_gain = np.max(total_pnl)

        fig.update_layout(
            title=f"PnL Diagram<br>Max Loss: {max_loss:,.2f}, Max Gain: {max_gain:,.2f}",
            xaxis_title="Spot Price",
            yaxis_title="PnL (Payoff - Cost)",
            template="plotly_white"
        )
        fig.show()

    def explain_pnl_at(self, spot: float, label: str = None):
        if label:
            print(f"\n-- Details at {label} Spot = {spot:,.2f} --")
        else:
            print(f"\n-- PnL Summary at Spot = {spot:,.2f} --")

        total_pnl = 0
        for leg in self.legs:
            payoff = leg.payoff(np.array([spot]))[0] * self.exchange_rate
            pnl = leg.pnl(np.array([spot]))[0] * self.exchange_rate
            qty = leg.quantity if hasattr(leg, 'quantity') else 0
            price = getattr(leg, 'price', 0) * self.exchange_rate
            cost = leg.total_cost() * self.exchange_rate
            print(f"{leg.description()}: "
                  f"Qty={qty:,}, Price={price:,.2f}, "
                  f"Cost={'-' if cost < 0 else ''}{abs(cost):,.2f}")
            print(f"  ↳ Payoff: {payoff:,.2f}, PnL: {pnl:,.2f}")
            total_pnl += pnl

        print(f"Total PnL: {total_pnl:,.2f}")
        
        
    def summary(self, spot_prices):
        print("---- Portfolio Summary ----")
        
        if self.exchange_rate != 1:
            print(f"\nExchange rate of {self.exchange_rate} is applied in the calculation\n")
        
        total_cost = sum(leg.total_cost() for leg in self.legs) * self.exchange_rate
        total_pnl_array = self.total_pnl(spot_prices) * self.exchange_rate

        max_pnl = np.max(total_pnl_array)
        min_pnl = np.min(total_pnl_array)

        tol = 1e-4
        spot_max_indices = np.where(np.abs(total_pnl_array - max_pnl) < tol)[0]
        spot_min_indices = np.where(np.abs(total_pnl_array - min_pnl) < tol)[0]

        spot_max_range = (spot_prices[spot_max_indices[0]], spot_prices[spot_max_indices[-1]])
        spot_min_range = (spot_prices[spot_min_indices[0]], spot_prices[spot_min_indices[-1]])

        print(f"Net Premium (Total Cost): {total_cost:,.2f}")

        if spot_max_range[0] == spot_max_range[1]:
            print(f"Maximum Gain: {max_pnl:,.2f} at Spot = {spot_max_range[0]:,.2f}")
        else:
            print(f"Maximum Gain: {max_pnl:,.2f} from Spot = {spot_max_range[0]:,.2f} to {spot_max_range[1]:,.2f}")

        if spot_min_range[0] == spot_min_range[1]:
            print(f"Maximum Loss: {min_pnl:,.2f} at Spot = {spot_min_range[0]:,.2f}")
        else:
            print(f"Maximum Loss: {min_pnl:,.2f} from Spot = {spot_min_range[0]:,.2f} to {spot_min_range[1]:,.2f}")

        self.explain_pnl_at(spot_prices[spot_min_indices[0]], label="Max Loss")
        self.explain_pnl_at(spot_prices[spot_max_indices[0]], label="Max Gain")

        print("---------------------------\n")


# Example usage:
spot_prices = np.linspace(0, 100, 500)

portfolio = Portfolio()
portfolio.add_leg(OptionLeg(strike=26, price=2.7, quantity=100000, option_type='put'))
portfolio.add_leg(OptionLeg(strike=30, price=6.2, quantity=-50000, option_type='put'))
portfolio.add_leg(StockLeg(price=24, quantity=50000))

portfolio.summary(spot_prices)

portfolio.plot_pnl(spot_prices)


---- Portfolio Summary ----
Net Premium (Total Cost): 1,160,000.00
Maximum Gain: 3,840,000.00 at Spot = 100.00
Maximum Loss: -60,000.00 from Spot = 0.00 to 25.85

-- Details at Max Loss Spot = 0.00 --
Long PUT @ 26: Qty=100,000, Price=2.70, Cost=270,000.00
  ↳ Payoff: 2,600,000.00, PnL: 2,330,000.00
Short PUT @ 30: Qty=-50,000, Price=6.20, Cost=-310,000.00
  ↳ Payoff: -1,500,000.00, PnL: -1,190,000.00
Long Stock @ 24.00: Qty=50,000, Price=24.00, Cost=1,200,000.00
  ↳ Payoff: 0.00, PnL: -1,200,000.00
Total PnL: -60,000.00

-- Details at Max Gain Spot = 100.00 --
Long PUT @ 26: Qty=100,000, Price=2.70, Cost=270,000.00
  ↳ Payoff: 0.00, PnL: -270,000.00
Short PUT @ 30: Qty=-50,000, Price=6.20, Cost=-310,000.00
  ↳ Payoff: -0.00, PnL: 310,000.00
Long Stock @ 24.00: Qty=50,000, Price=24.00, Cost=1,200,000.00
  ↳ Payoff: 5,000,000.00, PnL: 3,800,000.00
Total PnL: 3,840,000.00
---------------------------



In [67]:
# Example usage:
spot_prices = np.linspace(0, 100, 500)
exchange_rate = 1.1346
portfolio = Portfolio(exchange_rate=exchange_rate)
portfolio.add_leg(OptionLeg(strike=26, price=1.6291, quantity=100000, option_type='put'))
portfolio.add_leg(OptionLeg(strike=30, price=5.1736, quantity=-69600, option_type='put'))
portfolio.add_leg(StockLeg(price=24.88, quantity=30400))

portfolio.summary(spot_prices)
portfolio.plot_pnl(spot_prices)

---- Portfolio Summary ----

Exchange rate of 1.1346 is applied in the calculation

Net Premium (Total Cost): 634,444.99
Maximum Gain: 2,814,739.01 at Spot = 100.00
Maximum Loss: -53,529.79 from Spot = 0.00 to 25.85

-- Details at Max Loss Spot = 0.00 --
Long PUT @ 26: Qty=100,000, Price=1.85, Cost=184,837.69
  ↳ Payoff: 2,949,960.00, PnL: 2,765,122.31
Short PUT @ 30: Qty=-69,600, Price=5.87, Cost=-408,549.67
  ↳ Payoff: -2,369,044.80, PnL: -1,960,495.13
Long Stock @ 24.88: Qty=30,400, Price=28.23, Cost=858,156.98
  ↳ Payoff: 0.00, PnL: -858,156.98
Total PnL: -53,529.79

-- Details at Max Gain Spot = 100.00 --
Long PUT @ 26: Qty=100,000, Price=1.85, Cost=184,837.69
  ↳ Payoff: 0.00, PnL: -184,837.69
Short PUT @ 30: Qty=-69,600, Price=5.87, Cost=-408,549.67
  ↳ Payoff: -0.00, PnL: 408,549.67
Long Stock @ 24.88: Qty=30,400, Price=28.23, Cost=858,156.98
  ↳ Payoff: 3,449,184.00, PnL: 2,591,027.02
Total PnL: 2,814,739.01
---------------------------



In [68]:
portfolio.explain_pnl_at(30.26052)


-- PnL Summary at Spot = 30.26 --
Long PUT @ 26: Qty=100,000, Price=1.85, Cost=184,837.69
  ↳ Payoff: 0.00, PnL: -184,837.69
Short PUT @ 30: Qty=-69,600, Price=5.87, Cost=-408,549.67
  ↳ Payoff: -0.00, PnL: 408,549.67
Long Stock @ 24.88: Qty=30,400, Price=28.23, Cost=858,156.98
  ↳ Payoff: 1,043,741.01, PnL: 185,584.03
Total PnL: 409,296.02
